In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
import lightgbm as lgb
import numpy as np

from art.attacks import ZooAttack
from art.classifiers import LightGBMClassifier
from art.utils import load_mnist

In [ ]:
from pathlib import Path
import pickle

In [ ]:
(x_train, y_train), (x_test, y_test), _,_ = load_mnist()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 28 * 28))
x_test = x_test.reshape((x_test.shape[0], 28 * 28))

In [ ]:
params = {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 10}
train_set = lgb.Dataset(x_train, label=np.argmax(y_train, axis=1))
test_set = lgb.Dataset(x_test, label=np.argmax(y_test, axis=1))

In [ ]:
model = lgb.train(params=params, train_set=train_set, num_boost_round=100, valid_sets=[test_set])

In [ ]:
classifier = LightGBMClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value))

In [ ]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

In [ ]:
attack = ZooAttack(classifier=classifier, confidence=0.5, targeted=False, learning_rate=1e-1, max_iter=200,
                   binary_search_steps=100, initial_const=1e-1, abort_early=True, use_resize=False,
                   use_importance=False, nb_parallel=250, batch_size=1, variable_h=0.01)
x_test_adv = attack.generate(x=x_test)

In [ ]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))